### US Elections Data - ANLY 503 Final Exam

Collects the following files:   
    
**State Level Results**
- 'data/2016-president.csv'

**Voter Turnout from US Census**
- 'data/voter_turnout_age.csv'
- 'data/voter_turnout_demos.csv'
- 'data/non_voters.csv'

**Electoral College**
- 'data/'electoral_college.csv'

#### Prep

In [1]:
# set working directory
import os
path = '/Users/kgedney/Documents/georgetown/anly503/exam/'
os.chdir(path)

In [2]:
import pandas as pd

## Part 1: Results Data

#### 2016 Presidential by State

In [ ]:
# download from: https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/42MVDX/ZBRZDY&version=2.1

In [5]:
df_2016 = pd.read_csv('1976-2016-president.csv')

In [6]:
# subset on 2016
df_2016 = df_2016[df_2016['year'] == 2016].reset_index(drop=True)

In [7]:
# drop other candidates
df_2016 = df_2016[(df_2016['party'] == 'republican') | (df_2016['party'] == 'democrat')]
df_2016 = df_2016[df_2016['writein'] == False]

In [6]:
# save file
df_2016.to_csv('2016-president.csv', index=False)

#### Reshape for Leaflet Map

In [9]:
# subset
df_sub = df_2016[['state', 'totalvotes', 'candidate', 'candidatevotes']]

In [10]:
df_sub.head()

,state,totalvotes,candidate,candidatevotes
0,Alabama,2123372,"Trump, Donald J.",1318255
1,Alabama,2123372,"Clinton, Hillary",729547
5,Alaska,318608,"Trump, Donald J.",163387
6,Alaska,318608,"Clinton, Hillary",116454
12,Arizona,2573165,"Trump, Donald J.",1252401


In [11]:
# pivot to make columns for each candidate
z = df_sub.pivot(columns='candidate', index='state', values='candidatevotes')
z = z.reset_index()
z.head()

candidate,state,"Clinton, Hillary","Trump, Donald J."
0,Alabama,729547,1318255
1,Alaska,116454,163387
2,Arizona,1161167,1252401
3,Arkansas,380494,684872
4,California,8753788,4483810


In [12]:
# merge with total votes data
totalvotes = df_sub[['state', 'totalvotes']].drop_duplicates()

z = pd.merge(z, totalvotes, on='state').reset_index(drop=True)
z.head()

,state,"Clinton, Hillary","Trump, Donald J.",totalvotes
0,Alabama,729547,1318255,2123372
1,Alaska,116454,163387,318608
2,Arizona,1161167,1252401,2573165
3,Arkansas,380494,684872,1130635
4,California,8753788,4483810,14181595


In [13]:
# clean up
z.columns = ['state', 'clinton_votes', 'trump_votes', 'total_votes']

In [14]:
# add columns
z['clinton_pct'] = z['clinton_votes'] / z['total_votes']
z['trump_pct']   = z['trump_votes'] / z['total_votes']


In [23]:
z['points_diff'] = (z['trump_pct'] -  z['clinton_pct']) * 100 

In [25]:
z.head()

,state,clinton_votes,trump_votes,total_votes,clinton_pct,trump_pct,points_diff
0,Alabama,729547,1318255,2123372,0.343579,0.620831,27.725147
1,Alaska,116454,163387,318608,0.365509,0.512815,14.730641
2,Arizona,1161167,1252401,2573165,0.451260,0.486716,3.545595
3,Arkansas,380494,684872,1130635,0.336531,0.605741,26.920978
4,California,8753788,4483810,14181595,0.617264,0.316171,-30.109293


In [27]:
# save file
z.to_csv('2016-president-by-candidate.csv', index=False)

## Part 2:  Voter Turnout Data
Reference: https://electionlab.mit.edu/research/voter-turnout

#### 1. Who Votes? Source: US Census Bureau Surveys

In [86]:
# import data
url   = 'https://www2.census.gov/programs-surveys/cps/tables/p20/580/table04c.xlsx' # states by age
url_2 = 'https://www2.census.gov/programs-surveys/cps/tables/p20/580/table04b.xlsx' # states by race
url_3 = 'https://www2.census.gov/programs-surveys/cps/tables/p20/580/table10.xlsx'  # 

df_age  = pd.read_excel(url)
df_race = pd.read_excel(url_2) 
df_nonvoters = pd.read_excel(url_3)

In [95]:
# set column names
df_age.columns = ['state', 'age', 'total_pop', 'total_citizen_pop', 'total_registered', 'pct_registered','me_1', 'pct_citizen_regisered',
             'me2', 'total_vted', 'pct_voted', 'me3', 'pct_citizen_voted', 'me4', 'extra']

df_race.columns = ['state', 'demo', 'total_pop', 'total_citizen_pop', 'total_registered', 'pct_registered','me_1', 'pct_citizen_regisered',
             'me2', 'total_vted', 'pct_voted', 'me3', 'pct_citizen_voted', 'me4', 'extra']

In [96]:
# drop extra columns
df_age  = df_age.drop(columns=['me_1', 'me2', 'me3', 'me4', 'extra'])
df_race = df_race.drop(columns=['me_1', 'me2', 'me3', 'me4', 'extra'])

In [97]:
# drop extra rows
df_age  = df_age.drop([0,1,2,3])
df_race = df_race.drop([0,1,2,3])

In [98]:
# fill down state values
df_age['state']   = df_age['state'].ffill()
df_race['state']  = df_race['state'].ffill()

In [99]:
df_age.head(10)

,state,age,total_pop,total_citizen_pop,total_registered,pct_registered,pct_citizen_regisered,total_vted,pct_voted,pct_citizen_voted
4,US,Total,245502,224059,157596,64.2,70.3,137537,56,61.4
5,US,18 to 24,29320,26913,14905,50.8,55.4,11560,39.4,43
6,US,25 to 34,43794,38283,24682,56.4,64.5,20332,46.4,53.1
7,US,35 to 44,39905,34327,23948,60,69.8,20662,51.8,60.2
8,US,45 to 64,83799,77544,57394,68.5,74,51668,61.7,66.6
9,US,65+,48684,46993,36667,75.3,78,33314,68.4,70.9
10,ALABAMA,Total,3717,3651,2526,68,69.2,2095,56.4,57.4
11,ALABAMA,18 to 24,513,497,283,55.2,57,212,41.4,42.7
12,ALABAMA,25 to 34,583,568,393,67.5,69.2,310,53.1,54.5
13,ALABAMA,35 to 44,566,548,363,64.1,66.3,306,54,55.8


In [32]:
# save files
df_age.to_csv('voter_turnout_age.csv', index=False)
df_race.to_csv('voter_turnout_demos.csv', index=False)

In [87]:
# process other file
# set column names
df_nonvoters.columns = ['demo_1', 'demo_2', 'total_count', 'Illness or disability', 'Out of town', 
                        'Forgot to vote', 'Not interested', 'Too busy, conflicting schedule',
                         'Transportation problems', 'Did not like candidates or campaign issues', 
                        'Registration problems', 'Bad weather conditions', 'Inconvenient polling place', 
                        'Other reason', "Don't know or refused"]

In [88]:
# drop extra rows
df_nonvoters  = df_nonvoters.drop([0,1,2,3,4]).reset_index(drop=True)

In [89]:
# fill down demo_1 column
df_nonvoters['demo_1'] = df_nonvoters['demo_1'].ffill()

In [91]:
# drop margin of error rows
df_nonvoters  = df_nonvoters.drop(df_nonvoters.index[41:110])

In [93]:
# save file
df_nonvoters.to_csv('non_voters.csv', index=False)

##### Reshape for Tableau

In [40]:
df_nonvoters = pd.read_csv('non_voters.csv')

In [45]:
pd.melt(df_nonvoters, id_vars=['demo_1', 'demo_2'])

,demo_1,demo_2,variable,value
0,TOTAL,,total_count,18933
1,Age,18 to 24 years,total_count,3042
2,Age,25 to 44 years,total_count,7246
3,Age,45 to 64 years,total_count,5420
4,Age,65 years and over,total_count,3225
5,Sex,Male,total_count,9366
6,Sex,Female,total_count,9567
7,Race and Hispanic origin,White alone,total_count,14766
8,Race and Hispanic origin,White non-Hispanic alone,total_count,12633
9,Race and Hispanic origin,Black alone,total_count,2645


In [47]:
df_nonvoters.columns

Index(['demo_1', 'demo_2', 'total_count', 'Illness or disability',
       'Out of town', 'Forgot to vote', 'Not interested',
       'Too busy, conflicting schedule', 'Transportation problems',
       'Did not like candidates or campaign issues', 'Registration problems',
       'Bad weather conditions', 'Inconvenient polling place', 'Other reason',
       'Don't know or refused'],
      dtype='object')

In [52]:
df_nonvoters_melted = pd.melt(df_nonvoters, id_vars=['demo_1', 'demo_2', 'total_count'], value_vars = ['Illness or disability',
       'Out of town', 'Forgot to vote', 'Not interested',
       'Too busy, conflicting schedule', 'Transportation problems',
       'Did not like candidates or campaign issues', 'Registration problems',
       'Bad weather conditions', 'Inconvenient polling place', 'Other reason',
       "Don't know or refused"])

In [54]:
# save as excel
df_nonvoters_melted.to_csv('df_nonvoters_melted.csv', index=False, encoding='utf-8')

#### 2. How Many Vote? Source: US Elections Project

In [ ]:
# http://www.electproject.org/home/voter-turnout/voter-turnout-data

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1VAcF0eJ06y_8T4o2gvIL4YcyQy8pxb1zYkgXF76Uu1s/export?gid=2030096602&format=csv'

In [ ]:
df = pd.read_csv(url)

### Electoral College Data

In [33]:
# college electoral college points per state (source: kshaffer)
df = pd.read_csv('https://github.com/kshaffer/election2016/raw/master/2016ElectionAnalysis.csv')

In [35]:
df = df[['state', 'clintonElectors', 'trumpElectors', 'electors2016']]

In [37]:
df.to_csv('electoral_college.csv', index=False)

#### Electoral College Data for Tableau

In [28]:
df = pd.read_csv('https://github.com/kshaffer/election2016/raw/master/2016ElectionAnalysis.csv')

In [29]:
df = df[['state', 'clintonElectors', 'trumpElectors', 'electors2016']]

In [30]:
df = df.merge(z, on='state')

In [31]:
df.to_csv('electoral_college_tableau.csv')